<a href="https://colab.research.google.com/github/XinyaoTian/COMP5318_assignment2/blob/master/KmeansClustring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Environment setup
# Import assignment-wide packages
import sklearn
import numpy as np
import os
import pandas as pd
np.random.seed(42)

from random import uniform
import math

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")






In [31]:
# Load TripAdviosr dataset from its original source
TA_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00484/tripadvisor_review.csv", sep=',', encoding="utf-8")

# It's obviously that 'User ID' is not meaningful when clustering
# So we delete that column
TA_data = TA_data.drop('User ID', 1)

# Have a brief view
TA_data.head()

,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10
0,0.93,1.8,2.29,0.62,0.80,2.42,3.19,2.79,1.82,2.42
1,1.02,2.2,2.66,0.64,1.42,3.18,3.21,2.63,1.86,2.32
2,1.22,0.8,0.54,0.53,0.24,1.54,3.18,2.80,1.31,2.50
3,0.45,1.8,0.29,0.57,0.46,1.52,3.18,2.96,1.57,2.86
4,0.51,1.2,1.18,0.57,1.54,2.02,3.18,2.78,1.18,2.54


In [0]:
class KMeansClustering():

  def __init__(self, Data, k_numbers=1, similarity='euclidean', convergence_threshold = 0.01):
    self.Data = Data
    self.k = k_numbers
    self.similarity = similarity
    self.centroids = None
    self.DataDistributed = None
    self.convergence_threshold = convergence_threshold
  

  # initate k numbers of centroids
  def generate_initial_centroids(self):
    # automatically generate initial centroids
    i = 0
    centroids = list()
    while(i < self.k):
      centroid = dict()
      for column in self.Data.columns:
        # randomly generate a value in the range of column
        centroid[column] = float("{:.4f}".format(uniform(self.Data[column].min(0), self.Data[column].max(0))))
      centroids.append(centroid)
      i += 1
    # after generating centroids
    self.centroids = pd.DataFrame(centroids)
    return self.centroids


  # assign points to a cluster located by a centroid
  def assign_points(self):
    # create a list to store the closest distance
    distance_list = list()
    cluster_index = list()
    # iteration of rows(each point) in dataset
    for index_point, point in self.Data.iterrows():
      closest_distance = -1
      distributed_centroid = -1
      # for each point, calculate the distance from it to a centroid
      for index_centroid, centroid in self.centroids.iterrows():
        # ------ select distance type ------#
        # select distance type
        if self.similarity == "euclidean":
          distance = self.euclidean_distance(point, centroid)
        elif self.similarity == "manhattan":
          distance = self.manhattan_distance(point, centroid)
        elif self.similarity == "minkowski":
          distance = self.minkowski_distance(point, centroid)
        elif self.similarity == "cosine":
          distance = self.cosine_distance(point, centroid)
        else:
          # throw out a warning
          distance = -1
        # ------ -------------------- ------#
        # judge if new distance is closer than existing one
        if closest_distance == -1 or distance < closest_distance:
          # if it's true then update distance
          closest_distance = distance
          distributed_centroid = index_centroid
      # append distance to storeage list
      distance_list.append(closest_distance)
      cluster_index.append(distributed_centroid)
    # Add the cloest distance as a column to Dataframe
    # Using copy() to avoid build a pointer between Data and DataDistributed
    self.DataDistributed = self.Data.copy()
    self.DataDistributed['Distance'] = distance_list
    self.DataDistributed['Cluster_index'] = cluster_index
    return self.DataDistributed


  # generate empty centroid data structure
  def generate_empty_centroids(self):
    # automatically generate initial centroids
    i = 0
    centroids = list()
    while(i < self.k):
      centroid = dict()
      for column in self.Data.columns:
        # randomly generate a value in the range of column
        centroid[column] = float(0.0)
      centroids.append(centroid)
      i += 1
    return pd.DataFrame(centroids)
  

  # recalculate the centroid
  def recalculate_centroid(self):
    # recalculate centroids groupby previous clusters
    # avoid Cluster_index become a new index
    new_centroids = self.DataDistributed.groupby(['Cluster_index'], as_index=False).mean().round(4)
    # drop middle-process columns
    self.centroids = new_centroids.drop('Cluster_index', 1).drop('Distance', 1)
    return self.centroids
  

  # main process of the whole training
  def fit(self):
    # randomly generate k centroids
    self.generate_initial_centroids()
    
    convergence = False
    # iterate until meet convergence condition
    while convergence == False:
      # assign points to these centroids
      self.assign_points()
      # take the previous centroids
      previous_centroids = self.centroids.copy()
      # recalculate the centroids
      self.recalculate_centroid()
      new_centroids = self.centroids.copy()

      # judge if meeting convergence condition
      for index, centroid in self.centroids.iterrows():
        convergence = True
        # ------ select distance type ------#
        if self.similarity == "euclidean":
          centroid_movement = self.euclidean_distance(previous_centroids.iloc[index], new_centroids.iloc[index])
        elif self.similarity == "manhattan":
          centroid_movement = self.manhattan_distance(previous_centroids.iloc[index], new_centroids.iloc[index])
        elif self.similarity == "minkowski":
          centroid_movement = self.minkowski_distance(previous_centroids.iloc[index], new_centroids.iloc[index])
        elif self.similarity == "cosine":
          centroid_movement = self.cosine_distance(previous_centroids.iloc[index], new_centroids.iloc[index])
        # ------ -------------------- ------#
        if centroid_movement >= self.convergence_threshold:
          convergence = False
      
      
  # euclidean distance
  def euclidean_distance(self, pointA, pointB):
    sum_of_square = 0.0
    for column in self.Data.columns:
      sum_of_square += (pointA[column] - pointB[column])**2
    euclidean_distance = math.sqrt(sum_of_square)
    return float("{:.4f}".format(euclidean_distance))


  # manhattan distance
  def manhattan_distance(self, pointA, pointB):
    sum_of_abs = 0.0
    for column in self.Data.columns:
      sum_of_abs += abs(pointA[column] - pointB[column])
    manhattan_distance = sum_of_abs
    return float("{:.4f}".format(manhattan_distance))


  # Minkowski distance
  def minkowski_distance(self, pointA, pointB):
    # get degree of features
    degree = len(self.Data.columns)
    sum_of_poly_item = 0.0
    for column in self.Data.columns:
      sum_of_poly_item += pow(float("{:.4f}".format(pointA[column] - pointB[column])), degree)
    minkowski_distance = pow(sum_of_poly_item, 1/degree)
    return float("{:.4f}".format(minkowski_distance))
  

  # Cosine distance
  def cosine_distance(self, pointA, pointB):
    sum_of_dot_product = 0.0
    sum_of_square_pointA = 0.0
    sum_of_square_pointB = 0.0
    for column in self.Data.columns:
      sum_of_dot_product += pointA[column] * pointB[column]
      sum_of_square_pointA += pow(pointA[column], 2)
      sum_of_square_pointB += pow(pointB[column], 2)
    norm_of_pointA = math.sqrt(sum_of_square_pointA)
    norm_of_pointB = math.sqrt(sum_of_square_pointB)
    cosine_distance = sum_of_dot_product / (norm_of_pointA * norm_of_pointB)
    return float("{:.4f}".format(cosine_distance))


  
    

  


kmeans = KMeansClustering(TA_data, k_numbers=3, similarity='manhattan')
kmeans.fit()


In [21]:
class KMeansClustering():

  def __init__(self, Data, k_numbers=1, similarity='euclidean'):
    self.Data = Data
    self.k_numbers = k_numbers
    self.similarity = similarity
  
  def generate_initial_centroids(self):
    print(self.Data['Category 1'].max(0))
    

  


kmeans = KMeansClustering(TA_data)
kmeans.generate_initial_centroids()

3.22
